# Import some libraries

To run a code cell, select it then press CTRL-return

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from numpy import log, log10, sqrt, logspace

# Part I: Plot initial screening data of compounds to choose those for IC50 calculation

## Read in the initial screening data

The code below reads in the screening data and prints it out.   
The data are in long or tidy format where each column represents a single variable, and each row represents a single observation.  
The variables are as follows:

- **compound**: the names of the screened compounds and the controls
- **Male gamete function**: assay recorded as a percentage

There is also an index column running from 0 to 35 which can be ignored.

In [ ]:
compounds = pd.read_excel('Data/compounds.xlsx')
compounds

## Plot the data 

We want to plot the data to see what it looks like. We will use the seaborn module to do this as 1) you've used it before and 2) it makes nice plots with the minimal of code.

There are different ways of plotting the data: bar plot, box plot, swarm plot, strip plot and violin plot. For example, to make a bar plot you would code

```python
sns.barplot(data=compounds, y='compound', x='Male gamete function');
```
which will plot the compounds and controls on the *y*-axis and horizontal bars representing the mean Male gamete function for each compound.

In [ ]:
# plot the data


You can combine different types of plots in the same figure by just coding the plots in the same code cell. For example, this code will plot bars and individual data points.

```python
sns.barplot(data=compounds, y='compound', x='Male gamete function')
sns.stripplot(data=compounds, y='compound', x='Male gamete function');

```
Try different combinations to see which you think best summarises the data.

If you want to explore how else seaborn can plot categorical data, visit the official [website](https://seaborn.pydata.org/tutorial/categorical.html "Seaborn website").

In [ ]:
# make combinations of plots


### Based on your plot, which three compounds would you take forward for IC50 analysis?

---

# Part II: Estimating IC50 from dose-response curves 

## Read in the data

The code below reads in the dose-response data and prints out the first 5 and last 5 rows.   
The data are in long or tidy format.  
The meaning of the column names are as follows:

- **drug**: the names of the candidate drugs: HBX41108, NSC632839 and P22077
- **replicate**: biological replicates 1 to 5 for each drug
- **molarity**: drug dose (M)
- **response**: male gamete function as a percentage 

In [ ]:
assays = pd.read_excel('Data/drug_assays.xlsx')
assays

## Plot the dose-response curves

Next we plot the dose-response curves for each drug. Molarity is on the $x$-axis and response on the $y$-axis. To plot a curve for each drug we include `hue='drug'` in the plotting function.

- `sns.scatterplot` plots the individual responses for each replicate for each drug 
- `sns.lineplot` plots a line representing the average for each drug 

A scatter plot is useful for checking that the raw data look okay. A lineplot is useful for summarising the data. 

In [ ]:
# A combination of line and scatter plots
ax = sns.scatterplot(data=assays, x='molarity', y='response', hue='drug', legend=False)
sns.lineplot(data=assays, x='molarity', y='response', hue='drug', err_style=None)

# set the x-axis to a log scale because molarity varies across several orders of magnitude
ax.set_xscale('log')

# draw a horizontal line at 50%
ax.axhline(50, linestyle=':');

## Estimate by eye the IC50s of the three drugs

IC50 is the molarity at half-maximum, i.e., molarity when the response equals 50%. You are going to estimate these formally below using data-transformation and curve fitting procedures. But first, as always when working with data, it is useful to try and estimate things by eye. Why is it useful? Because it is a good sanity check that your formal procedures are giving you the answers you expect. It is also useful because it forces you think about and understand your data.

Try and estimate by eye the IC50s of the three drugs. Don't worry if they are not precise - you just want ball-park numbers. Write your answers in the cell below.

> IC50 estimates by eye for each drug

## Log-transform the data

As with most datasets, the raw data have to be transformed before it can be formally analysed.

1. The drug concentrations vary over several orders of magnitude: from $10^{-9}$ M to $10^{-5}$ M. These must be transformed onto a log-scale so that the data points are evenly spaced along the $x$-axis as in the plot above. This is done by applying a log-transform.
2. The response data need to be transformed from a percentage (0-100) to a proportion (0-1). This is simply done by dividing the response by 100.

After transforming the data we should replot it.

In [ ]:
# tranform the raw data
assays['log10_molarity'] = log10(assays['molarity'])
assays['response_proportion'] = assays['response'] / 100

# plot the transformed data
ax = sns.scatterplot(data=assays, x='log10_molarity', y='response_proportion', hue='drug', legend=False)
sns.lineplot(data=assays, x='log10_molarity', y='response_proportion', hue='drug', err_style=None);

# draw a horizontal line at 0.5
ax.axhline(0.5, linestyle=':');

The plot should look almost the same as the one above, with two differences:
1. The *y*-axis scale is from 0 to 1 rather than 0 to 100.
2. The *x*-axis runs from -9 to -5 rather than from $10^{-9}$ to $10^{-5}$ because we have log-transformed molarity.

## One more transformation

There is one final transformation to make. Notice that log10_molarity is negative (-9 to -5). To fit a curve to this data (in order to estimate IC50) the *x*-axis values must be positive.

We can simply add the value of 9 to the log10_molarity values to make all the values positive.

This is done in the code cell below, and re replot to make sure the transformation has worked as intended.

In [ ]:
# add 9 to log10 molarity to make all values positive
assays['log10_molarity'] = log10(assays['molarity']) + 9

# plot the transformed data
ax = sns.scatterplot(data=assays, x='log10_molarity', y='response_proportion', hue='drug', legend=False)
sns.lineplot(data=assays, x='log10_molarity', y='response_proportion', hue='drug', err_style=None);
ax.set_xlabel('9 + log10(molarity)');

## Transformed dose-response curves are described by a Hill function

Dose-response curves often have an S-shape (called a sigmoidal shape). The Hill function is the name of a curve that has such a shape. It's formula is

$$y=\frac{k^n}{k^n+x^n}$$

([Desmos](https://www.desmos.com/calculator) is an excellent online curve-plotting app.)

In this formula $x$ represents transformed molarity ($x=9+\log_{10}(\text{molarity})$) and $y$ represents the transformed response ($y=\frac{\text{response}}{100}$). 

The parameter $n$ is called the Hill coefficient. It determines how fast the curve falls. We are not interested in its value.

The parameter $k$ is the value of $x$ at half-maximum (i.e., when $y=\frac{1}{2}$). You can see this is so by substituting $x=k$ into the Hill function as follows:
$$
\begin{align*}
y &= \frac{k^n}{k^n+k^n} \\
y &= \frac{k^n}{2k^n} \\
y &= \frac{1}{2}
\end{align*}
$$

In other words, when $x=k$, $y=\frac{1}{2}$.


We are interested in estimating the drug concentration when the response is half of its maximum. This concentration is called IC50. Therefore the parameter $k$ is the transformed IC50.

What we will do below is fit the Hill function to the transformed data giving us an estimate of $k$. We then back-transform this value to find IC50. To find the back-transform formula remember that the transformed molarity $x$, is related to actual molarity by the formula

$$x=9+\log_{10}(\text{molarity})$$
Substituting in $x=k$ and $\text{molarity}=\text{IC50}$ we get
$$k=9+\log_{10}(\text{IC50})$$
Subtracting 9 from both sides:
$$k-9=\log_{10}(\text{IC50})$$
Raising to the power of 10 on both sides:
$$10^{k-9}=10^{\log_{10}(\text{IC50})}$$
Resulting in the back-transformation formula
$$10^{k-9}=\text{IC50}$$

So to estimate IC50 for a drug, we fit the transformed data to a Hill function to estimate a value for $k$ and then substitute that value into the back-transformation formula to find IC50.


First we have to code the Hill function. Run the code cell below. Although nothing will appear to happen, we have now defined the Hill function which we will use below to fit a Hill curve to the data.

In [ ]:
def hill(x, k, n):
    """ The hill function: n is the hill coefficient and x equals k at half-maximum"""
    return k**n / (k**n + x**n)

## Fit a Hill curve to the transformed data

Then we have to fit the curve to the data. In the code below we fit it to drug P22077.

In [ ]:
# HBX41108, NSC632839, P22077

# extract one drug's data from the dataframe and save to a new dataframe called d
d = assays.query('drug == "P22077"').copy()

# use curve_fit to fit the hill function to the transformed data in dataframe d
# the estimated parameters, k and n, are returned and stored in mu
# the squares of the standard errors of k and n (ie the uncertainty in their estimates) are returned and stored in cov
mu, cov = curve_fit(hill, d['log10_molarity'], d['response_proportion'])

## Output the estimates of $k$ and IC50

The estimated values of $k$ and $n$ are stored in the variable `mu`. 

The code below prints them to two decimal places.

In [ ]:
k = mu[0]
n = mu[1]

print(f'k = {k:.2f}')
print(f'n = {n:.2f}')

Back-transform $k$ to find IC50, and print to two significant figures.

In [ ]:
IC50 = 10**(k-9)
print(f'IC50 = {IC50:.2g} M')

If you're not familiar with computer e-notation the symbol `e` in a number means "times 10 to the power". So `1e-5` means $1\times 10^{-5}$ (i.e, one **times 10 to the power** -5).

Let's convert IC50 to the more convenient micromolar scale by multiplying by $10^6$:

In [ ]:
print(f'IC50 = {IC50 * 1e6:.1f} μM')

So the estimated IC50 for P22077 is 4.5 μM. 

Is this close to the value you estimated by eye?

## Calculate the uncertainty of the IC50 estimate (95% CI)

Whenever we estimate something we should also provide an estimate of our uncertainty in its value.

For that we use the second piece of information from the Hill function curve fit. The variable `cov` contains estimates of the uncertainties in the Hill function parameters $k$ and $n$. We're not interested in $n$ so we'll ignore this. The uncertainty in $k$ is stored in `cov[0][0]`.

For convenience, in the code cell below, we store `cov[0][0]` in a variable called `v_k`.

In [ ]:
v_k = cov[0][0]
v_k

The square-root of `v_k`, called the standard error (SE) of $k$, is a measure of the uncertainty in the estimate of $k$.

Standard error is usually given the symbol $\sigma$:

$$\sigma_k = \sqrt {v_k}$$

The uncertainty in the estimate of $k$ is reported in, usually, one of two ways:
- Either the estimate ± standard error
- Or the 95% confidence interval, which is the estimate ± 1.96 x standard error 

It doesn't matter which way you chose, as long as you report which one you used so that someone reading your conclusions know how to interpret your conclusions.

In [ ]:
# the standard error of k
sigma_k = sqrt(v_k)

# the lower and upper bounds of the 95% CI of k
k_lower = k - 1.96 * sigma_k
k_upper = k + 1.96 * sigma_k

print(f'estimate of k is {k:.2f} ± {sigma_k:.2f} (SE)')
print(f'estimate of k is {k:.2f}, 95% CI = ({k_lower:.2f}, {k_upper:.2f})')

The standard error in the IC50 estimate is given by the formula

$$\text{standard error of IC50} = \text{IC50} \times \text{standard error of } k \times \log_e(10)$$
 
Or, using symbols

$$\sigma_\text{IC50} = \text{IC50} \times \sigma_k \times \log_e(10)$$

In [ ]:
# the standard error of IC50
sigma_IC50 = IC50 * sigma_k * log(10)

# the lower and upper bounds of the 95% CI of IC50
IC50_lower = IC50 - 1.96 * sigma_IC50
IC50_upper = IC50 + 1.96 * sigma_IC50

print(f'estimate of IC50 is {IC50:.2g} ± {sigma_IC50:.2g} M (SE)')
print(f'estimate of IC50 is {IC50:.2g} M, 95% CI = ({IC50_lower:.2g}, {IC50_upper:.2g}) M')

Or on the micromolar scale

In [ ]:
print(f'estimate of IC50 is {IC50*1e6:.1f} ± {sigma_IC50*1e6:.1f} μM (SE)')
print(f'estimate of IC50 is {IC50*1e6:.1f} μM, 95% CI = ({IC50_lower*1e6:.1f}, {IC50_upper*1e6:.1f}) μM')

## Plot the fitted curve with the data

In [ ]:
# intialise a new figure
fig, ax = plt.subplots()

# molarity varies from 1e-9 to 1e-4 (see first figure above)
# create a list of 100 values evenly spaced from 1e-9 to 1e-4 on the log10 scale
molarity = logspace(start=-9, stop=-4, num=100, base=10)

# transformed molarity
x = 9 + log10(molarity)

# plot the fitted curve with molarity on a log scale
ax.plot(molarity, hill(x, k, n))
ax.set_xscale('log')

# plot the data
ax.scatter(data=d, x='molarity', y='response_proportion')

# add some annotation
ax.set_xlabel('Molarity (M)')
ax.set_ylabel('Male gamete function')
ax.set_title('Dose-response of P22077')

# add a vertical band representing the 95% CI of IC50
ax.axvspan(IC50_lower, IC50_upper, alpha=0.25)
# add a dotted horizontal line at half-maximum (ie 0.5)
ax.axhline(0.5, linestyle=':');

## Now it's your turn

Now calculate IC50s with 95% CIs for the other two drugs, and plot their fits to the dose-response data.

---